# Model Context Protocol

[Model Context Protocol (MCP)](https://modelcontextprotocol.io/docs/getting-started/intro) is an open-source standard for connecting AI applications to external systems.

External systems can include:

+ Connecting and querying data sources like local files and databases.
+ Tools such as search engines and calculators.
+ Workflows, including specialized prompts.

MCP provides a standard interface for AI applications.

![](./img/06_mcp.png)

The key participants in the MCP architecture are:

+ MCP Host: The AI application that coordinates and manages one or multiple MCP clients
+ MCP Client: A component that maintains a connection to an MCP server and obtains context from an MCP server for the MCP host to use
+ MCP Server: A program that provides context to MCP clients

For example, VS Code acts as an MCP host. When VS Code connects to an MCP server, like the Sentry MCP server, the VS Code runtime instantiates an MCP client objects that maintains a connection to the Sentry MCP server.

# MCP Server

The library [fastmcp](https://gofastmcp.com/getting-started/welcome) allows us to create mcp servers.

The code below, also found in `./05_src/static_mcp/server.py`, instantiates an MCP server.


```python
from fastmcp import FastMCP

mcp = FastMCP("My MCP Server")

@mcp.tool
def greet(name: str) -> str:
    return f"Hello, {name}!"

if __name__ == "__main__":
    mcp.run(
        transport="http",
        host="localhost", 
        port=3000, 
    )
```

## Reverse Proxy

The OpenAI SDK requires an https connection to any MCP server. To run the MCP server locally, we may need a TLS certificate provided by a trusted authority. To satisfy this requirement, we use a service called [ngrok](https://ngrok.com/). Among other services, ngrok allows us to use a URL for connecting with HTTPS. It solves the issue of provisioning a TLS certificate from a trusted authority. 


# Static MCP

In a terminal with the working directory as `./05_src/`: 

+ Use `python -m static_mcp.server` to start the server.
+ Use `ngrok http 3000` to set up the reverse proxy.

All calls to the URL in the environment variable `MCP_URL` will be directed to localhost:3000. `MCP_URL` is what ngrok reports after using the second command.

In [ ]:
%load_ext dotenv
%dotenv ../../05_src/.env
%dotenv ../../05_src/.secrets

In [ ]:
import os
from openai import OpenAI
client = OpenAI()

mcp_url = os.getenv("MCP_URL")

print(f'Using MCP URL: {mcp_url}')

tools = [
    {
        "type": "mcp",
        "server_label": "greeting_service",
        "server_description": "A greeting service that returns personalized greetings.",
        "server_url": mcp_url,
        "require_approval": "never",
    },
]


resp = client.responses.create(
    model="gpt-5",
    tools=tools,
    instructions="Use the greeting service to answer the question.",
    input="Hello, I am Alice.",
)

print(resp.output_text)

# Static Weather

A more descriptive setup, but still static mcp server can be started with module `static_weather_mcp.server`. Simply stop the previous server (CTRL+C) and start the second one.

In [ ]:

from openai import OpenAI
client = OpenAI()

mcp_url = os.getenv("MCP_URL")

_logs.info(f'Using MCP URL: {mcp_url}')

tools = [
    {
        "type": "mcp",
        "server_label": "weather_service",
        "server_description": "A weather service that returns current weather conditions.",
        "server_url": mcp_url,
        "require_approval": "never",
    },
]


resp = client.responses.create(
    model="gpt-5",
    tools=tools,
    instructions="Use the weather service to answer the question.",
    input="What is the weather currently?",
)

print(resp.output_text)